The first section of your notebook should include why you chose this data and data provenance.

Why I chose this data: 
    I chose this data set because I love the idea of anaylizing a large data set about book! The reviews of the data set are good and people seem to not have problems with it. I found a manipulated version of the origional data set where a Kaggle user scrapped the internet and added genres to the data set. This was the data set I origionally downloaded from Kaggle and later manipulated to form the data.csv file. Overall, the data appears to be trustworthy and accurate. It included genres and titles, which are two things I really want to focus on. I want to be able to possibly look up images based on an isbn number, so I wanted to have that information. I like the idea of being able to look up a list of books based on an author. I like the idea of being able to click on a book and getting a page of information on the book and different comparisons based on its genres. 

Data Provenance:

    ORIGIONAL DATA SET: https://www.kaggle.com/datasets/jealousleopard/goodreadsbooks
        - This is the origional data set that was created by scraping the Goodreads API under the CC0: Public Domain license. It is updated weekly. 

    MANIPULATED DATA SET by Kaggle user: https://www.kaggle.com/datasets/middlelight/goodreadsbookswithgenres/data
        - User added genres by making a call to 'https://www.googleapis.com/books/v1/volumes?q=isbn:' 
        Books for which a genre could not be found were dropped rather than manually added in
        
    PERSONAL MANIPULATIONS TO DATA SET: data.csv 
        - Removed all rows containing N/A values
        - Renamed column headers to follow personal formatting
        - Seperated Genres column into seperate columns for each genre. This was done with the intention of being able to analyze genres seperately. If a book was placed within the genre, it recieved a 1 and if it was not within the genre it recieved a 0. 
        - Next, genres with the same super genre were grouped together to lessen the total number of genres. 
        - Next, a column mapping genres that I determined were neccessary to the data set's genres was applied to the set and new genres were determined. 
        - Finally, I reordered some of the columns, so that they were in my desired order. 

In [20]:
# import dependencies

from dash import Dash, html, dcc 
import pandas as pd
import plotly.express as px

In [21]:
#read in csv file 
grdwg = pd.read_csv('Goodreads_books_with_genres.csv')
#grdwg.head()

In [22]:
#recieved general info about the origional data set
#grdwg.info()

In [23]:
#remove rows that contain N/A values
grdwg.dropna(inplace=True)

In [24]:
#grdwg.info()

In [25]:
#make sure there are no duplicate rows 
duplicate_rows = grdwg.duplicated()
print("number of duplicate rows:" , duplicate_rows.sum())

number of duplicate rows: 0


In [26]:
#rename columns that need to be lower case and follow the defined format
grdwg.rename(columns = {'Book Id':'book_id', 'Title': 'title',
                        'Author': 'author'}, inplace =True)

In [27]:
#checks the number of unique items in each column
#grdwg.nunique()

In [28]:
#I wanted to know the unique values in the genre column
#grdwg.genres.unique()

In [29]:
#splits the genres column values by ; and turns it into a list of the seperated items
grdwg['genres'] = grdwg['genres'].astype(str).str.split(';')
#this was just me wanting to see what the current different unique genres were
unique_genres = set(genre for sublist in grdwg['genres'] for genre in sublist)
#print(unique_genres)

In [30]:
#splits the genres into different columns
genres_df = grdwg['genres'].str.join('|').str.get_dummies()

#add the new columns to the origional dataframe
grdwg = pd.concat([grdwg, genres_df], axis=1)
#look at the rows 
for index, row in grdwg.iterrows():
    #look at all the genres in the list of genres
    for genre in row['genres']:
        #set the corresponding genre column to 1 if when it appears in the list 
        grdwg.at[index, genre] = 1
#i just wanted to see all of the column names
pd.set_option('display.max_columns', None)
grdwg.head()

,book_id,title,author,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,genres,1961-1975,40k,AUTOBIOGRAPHY,Academic,"Academic,Academia","Academic,Academics","Academic,College","Academic,Grad School","Academic,Read For School","Academic,School","Academic,Students",Action,Activities,Adolescence,Adult,Adult Fiction,"Adult Fiction,Erotica",Adventure,"Adventure,Maritime","Adventure,Pirates","Adventure,Survival","Africa,Eastern Africa","Africa,Northern Africa","African Literature,Egyptian Literature","Alcohol,Beer","Alcohol,Cocktails","Alcohol,Wine",American,"American Civil War,Civil War History","American Civil War,Civil War Western Theater","American History,American Civil War","American Revolution,American Revolutionary War",American fiction,American literature,"American,African American Literature","American,American Fiction","American,Americana","American,Southern",Amish,Anarchism,Animals,"Animals,Animal Fiction","Animals,Anthropomorphic","Animals,Birds","Animals,Cats","Animals,Dinosaurs","Animals,Dogs","Animals,Horses","Animals,Rabbits",Anthologies,"Anthologies,Collections",Anthropology,"Anthropology,Ethnography",Antiques,"Apocalyptic,Post Apocalyptic","Archaeology,Egyptology","Archaeology,Material Culture",Architecture,Art,Art and Photography,"Art,Art Design","Art,Art History","Art,Crafts","Art,Drawing","Art,Photography","Art,Visual Art","Art,Woodwork",Artificial Intelligence,"Asian Literature,Chinese Literature","Asian Literature,Indian Literature","Asian Literature,Indonesian Literature","Asian Literature,Japanese Literature","Asian Literature,Turkish Literature","Astronomy,Planets",Athens (Greece),Audiobook,Authors,Autobiography,"Autobiography,Memoir",Aviation,BIOGRAPHY,"Banking,Banks",Bears,"Beer,Brewing",Bibles,Biblical,Biblical Fiction,Biography,Biography Memoir,"Biography,Autobiography","Biography,Music Biography","Biology,Ecology","Biology,Evolution","Biology,Genetics","Biology,Neuroscience",Body,Book Club,Brothers and sisters,"Buddhism,Zen",Buisness,Business,"Business,Amazon","Business,Apple","Business,Entrepreneurship","Business,Management",Canon,"Cartography,Atlases","Cartography,Maps","Category Romance,Harlequin","Category Romance,Love Inspired","Category Romance,Loveswept","Category Romance,Silhouette","Category Romance,Traditional Regency","Central Africa,Democratic Republic Of The ...",Childrens,Childrens poetry,Childrens stories,"Childrens,1st Grade","Childrens,2nd Grade","Childrens,Chapter Books","Childrens,Childrens Classics","Childrens,Juvenile","Childrens,Middle Grade","Childrens,Nursery Rhymes","Childrens,Picture Books","Childrens,Pre K","Childrens,School Stories","Childrens,Storytime","Childrens,Young Readers",Chinese Americans,Christian,Christian Fiction,"Christian,Christian Living","Christian,Christian Non Fiction","Christianity,Catholic","Christianity,Church Ministry","Christianity,Discipleship","Christianity,Evangelism","Christianity,Lds","Christianity,New Testament","Christianity,Old Testament","Church,Church History","Cities,Urban Planning","Cities,Urban Studies","Cities,Urbanism","Civil War,Spanish Civil War",Classics,"Classics,Classic Literature","Classics,Cult Classics","Classics,Modern Classics",Collectibles,"Combat,Martial Arts","Comedy,Stand Up",Comics,Comics Manga,"Comics,Comic Book","Comics,Superheroes","Complementary Med...,Traditional Chinese Medicine","Computer Science,Algorithms","Computer Science,Coding","Computer Science,Computers","Computer Science,Programming","Computer Science,Software","Computer Science,Technical",Computer software,Computers,"Computers,Hackers","Computers,Internet",Conservation,Contemporary,"Contemporary Romance,Sports Romance",Cooking,"Cooking,Food Preservation","Couture,Fashion",Crafts,"Crafts,Beading","Crafts,Jewellery Making","Crafts,Knitting","Crafts,Quilting","Crafts,Sewing","Crime,True Crime",Criticism,"Criticism,Lit Crit","Criticism,Literary Criticism",Cultural,"Cultural,Africa","Cultural,African

In [31]:
#the purpose of this is to eliminate genre columns that basically have the same meaning. Something like Fantasy and Fantasy, childrens
#i just want those to be grouped into fantasy
#traverse through all of the columns
for col in grdwg.columns:
    #if there is a , in the column name
    if ',' in col:
        new_col_name = col.split(',')[0]  #then keep the part of the string that comes before the comma
        grdwg.rename(columns={col: new_col_name}, inplace=True) #rename the column that contained the comma
#see the results...now there are repeat column names...next step is to remove the repeats
grdwg.head()

,book_id,title,author,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,genres,1961-1975,40k,AUTOBIOGRAPHY,Academic,Academic,Academic,Academic,Academic,Academic,Academic,Academic,Action,Activities,Adolescence,Adult,Adult Fiction,Adult Fiction,Adventure,Adventure,Adventure,Adventure,Africa,Africa,African Literature,Alcohol,Alcohol,Alcohol,American,American Civil War,American Civil War,American History,American Revolution,American fiction,American literature,American,American,American,American,Amish,Anarchism,Animals,Animals,Animals,Animals,Animals,Animals,Animals,Animals,Animals,Anthologies,Anthologies,Anthropology,Anthropology,Antiques,Apocalyptic,Archaeology,Archaeology,Architecture,Art,Art and Photography,Art,Art,Art,Art,Art,Art,Art,Artificial Intelligence,Asian Literature,Asian Literature,Asian Literature,Asian Literature,Asian Literature,Astronomy,Athens (Greece),Audiobook,Authors,Autobiography,Autobiography,Aviation,BIOGRAPHY,Banking,Bears,Beer,Bibles,Biblical,Biblical Fiction,Biography,Biography Memoir,Biography,Biography,Biology,Biology,Biology,Biology,Body,Book Club,Brothers and sisters,Buddhism,Buisness,Business,Business,Business,Business,Business,Canon,Cartography,Cartography,Category Romance,Category Romance,Category Romance,Category Romance,Category Romance,Central Africa,Childrens,Childrens poetry,Childrens stories,Childrens,Childrens,Childrens,Childrens,Childrens,Childrens,Childrens,Childrens,Childrens,Childrens,Childrens,Childrens,Chinese Americans,Christian,Christian Fiction,Christian,Christian,Christianity,Christianity,Christianity,Christianity,Christianity,Christianity,Christianity,Church,Cities,Cities,Cities,Civil War,Classics,Classics,Classics,Classics,Collectibles,Combat,Comedy,Comics,Comics Manga,Comics,Comics,Complementary Med...,Computer Science,Computer Science,Computer Science,Computer Science,Computer Science,Computer Science,Computer software,Computers,Computers,Computers,Conservation,Contemporary,Contemporary Romance,Cooking,Cooking,Couture,Crafts,Crafts,Crafts,Crafts,Crafts,Crafts,Crime,Criticism,Criticism,Criticism,Cultural,Cultural,Cultural,Cultural,Cultural,Cultural,Cultural,Cultural,Cultural,Cultural,Cultural,Cultural,Cultural,Cultural,Cultural,Cultural,Cultural,Cultural,Cultural,Cultural,Cultural,Cultural,Cultural,Cultural,Cultural,Cultural,Cultural,Cultural,Cultural,Cultural,Cultural,Cultural,Cultural,Cultural,Cultural,Cultural,Cultural,Cultural,Culture,Culture,Culture,Culture,Currency,Dark,Dark Fantasy,Dc Comics,Dc Comics,Death,Design,Design,Design,Diary,Diary,Disability,Disciplines,Doctors,Drama,Dungeons and Dragons (Game),Dungeons and Dragons,Dungeons and Dragons,Earth,Eastern Africa,Eastern Africa,Eastern Africa,Eastern Africa,Eastern Africa,Eastern Africa,Eastern Africa,Economics,Economics,Education,Emergency Services,Emergency Services,Engineering,English History,Environment,Environment,Environment,Environment,Environment,Environment,Epic,Erotica,Erotica,Esoterica,Esoterica,Etiquette,European,European History,European Literature,European Literature,European Literature,European Literature,European Literature,European Literature,European Literature,European Literature,European Literature,European Literature,European Literature,European Literature,European Literature,European Literature,European Literature,European Literature,Fairies,Fairy Tales,Fairy Tales,Fairy Tales,Fairy Tales,Fairy Tales,Family,Family Law,Family Law,Fan Fiction,Fantasy,Fantasy Of Manners,Fantasy fiction,Fantasy,Fantasy,Fantasy,Fantasy,Fantasy,Fantasy,Fantasy,Fantasy,Fantasy,Fantasy,Fantasy,Fantasy,Fantasy,Fantasy,Fantasy,Fantasy,Fantasy,Female Authors,Feminism,Feminism,Fiction,Films and filming,Finance,Finance,Finance,Finance,Fitness,Folk Tales,Folk Tales,Folklore,Folklore,Food and Drink,Food and Drink,Food and Drink,Food and Drink,Food and Drink,Food and Drink,Food and Drink,Food and Drink,Food and Drink,Food and Drink,Food and Drink,Food and Drink,Food an

In [32]:
#group the columns with common names and add their corresponding row values together. (this ensures that if one of the columns
#contained a 1, then the final column will contain a positive integer)
grdwg = grdwg.groupby(grdwg.columns, axis=1).sum()
#look at the column names now 
grdwg.head(1)


,1961-1975,40k,AUTOBIOGRAPHY,Academic,Action,Activities,Adolescence,Adult,Adult Fiction,Adventure,Africa,African Literature,Alcohol,American,American Civil War,American History,American Revolution,American fiction,American literature,Amish,Anarchism,Animals,Anthologies,Anthropology,Antiques,Apocalyptic,Archaeology,Architecture,Art,Art and Photography,Artificial Intelligence,Asian Literature,Astronomy,Athens (Greece),Audiobook,Authors,Autobiography,Aviation,BIOGRAPHY,Banking,Bears,Beer,Bibles,Biblical,Biblical Fiction,Biography,Biography Memoir,Biology,Body,Book Club,Brothers and sisters,Buddhism,Buisness,Business,Canon,Cartography,Category Romance,Central Africa,Childrens,Childrens poetry,Childrens stories,Chinese Americans,Christian,Christian Fiction,Christianity,Church,Cities,Civil War,Classics,Collectibles,Combat,Comedy,Comics,Comics Manga,Complementary Med...,Computer Science,Computer software,Computers,Conservation,Contemporary,Contemporary Romance,Cooking,Couture,Crafts,Crime,Criticism,Cultural,Culture,Currency,Dark,Dark Fantasy,Dc Comics,Death,Design,Diary,Disability,Disciplines,Doctors,Drama,Dungeons and Dragons,Dungeons and Dragons (Game),Earth,Eastern Africa,Economics,Education,Emergency Services,Engineering,English History,Environment,Epic,Erotica,Esoterica,Etiquette,European,European History,European Literature,Fairies,Fairy Tales,Family,Family Law,Fan Fiction,Fantasy,Fantasy Of Manners,Fantasy fiction,Female Authors,Feminism,Fiction,Films and filming,Finance,Fitness,Folk Tales,Folklore,Food and Drink,Football,Frontier and pioneer life,Futurism,Futuristic,GLBT,Games,Gardening,Gay,Gender,Gender and Sexuality,Geography,Geology,German History,Good and evil,Gothic,Graphic Novels,Graphic Novels Comics,Graphic Novels Manga,Harlequin,Health,Heroic Fantasy,Historical,Historical Fiction,Historical Romance,History,History and Politics,Hobbies,Holiday,Home,Homeschool,Horror,Horror tales,Horticulture,House,How To,Hugo Awards,Humanities,Humor,Illness,India,Inspirational,International Rel...,Internet,Italian drama,Jewellery,Judaism,Juvenile Fiction,Juvenile Nonfiction,Kids,LGBT,LITERARY CRITICISM,Labor,Language,Language Arts,Law,Lds,Leadership,Lesbian,Library Science,Linguistics,Literary Collections,Literary Criticism,Literary Fiction,Literature,Love,Lovecraftian,Magical Realism,Manga,Marriage,Marvel,Mary Shelley,Mathematics,Media Tie In,Medical,Medical Professions,Medicine,Medievalism,Menage,Mental Health,Mental health.,Middle English Li...,Military,Military Fiction,Military History,Mind,Minimal brain dysfunction in children,Mmorpg,Modern,Mountaineering,Museology,Music,Mystery,Mythology,Native Americans,Nature,Neurodiversity,Neuroscience,New Adult,New York,Nobel Prize,Nonfiction,North American Hi...,Northern Africa,Novella,Novels,Number,Nursing,Occult,Outdoors,Own,Paranormal,Parenting,Performing Arts,Personal,Pets,Philosophy,Photography,Physics,Planets,Plants,Plays,Poetry,Poetry Plays,Political Science,Politics,Polyamory,Pop Culture,Pornography,Prayer,Presidents,Productivity,Pseudoscience,Psychology,Pulp,Queer,Race,Railways,Realistic Fiction,Recreation,Reference,Relationships,Religion,Reportage,Retellings,Role Playing Games,Roman,Romance,Romantic,Russian History,Scandinavian Lite...,School,School Stories,Sci Fi Fantasy,Science,Science Fiction,Science Fiction Fantasy,Science Nature,Self Help,Sequential Art,Sewing,Sex,Sex Work,Sexuality,Shapeshifters,Short Stories,Singularity,Soccer,Social,Social Issues,Social Justice,Social Movements,Social Science,Sociology,Southern Africa,Space,Spanish History,Speculative Fiction,Spirit,Spiritual life,Spirituality,Sports,Sports and Games,Spy Thriller,Star Trek,Stone age,Study Aids,Superheroes,Surreal,Suspense,Swedish,Teaching,Technology,Television,Textbooks,The Americas,The United States Of America,Thriller,Tragedy,Transport,Transportation,Travel,True Story,Unfinished,United States,Urban,Vegetarian,Vietnam War,War,Warriors,Weird Fiction,Western Africa,Westerns,Wildlife,Witchcraft,Womens,Womens

In [33]:
#This is my column mapping dictionary. I defined these genres myself. I chose the main genres from the goodreads website, 
#so that there would be a good amount of overlap. I simply wanted to decrease the total number of genres, so I grouped them 
#into larger umbrellas. This will make everything cleaner when generating visuals in the dashboard. 
column_mapping = {
    'adult': ['Adult', 'New Adult', 'Pornography', ],
    'art': ['Art','Architecture', 'Art and Photography', 'Couture', 'Design', 'Films and filming', 'Language Arts', 
             'Performing Arts', 'Photography', 'Sequential Art', 'Sewing', ],
    'audiobook':['Audiobook'],
    'biography': ['AUTOBIOGRAPHY','Autobiography','BIOGRAPHY','Biography', 'Diary'], 
    'business': ['Banking','Buisness', 'Business', 'Currency', 'Economics', 'Finance'],
    'childrens': ['Adolescence', 'Childrens', 'Childrens stories', 'Kids', ],
    'classics':['Classics', 'Mary Shelley',  ],
    'comics': ['Comics','Dc Comics','Graphic Novels Comics', 'Marvel', ],
    'comedy': ['Humor', ],
    'contemporary': ['Contemporary'],
    'culture':['Amish', 'Cultural', 'Culture', 'Language', ],
    'food_and_drink': ['Alcohol','Beer', 'Cooking', 'Food and Drink', 'Vegetarian', ],
    'comedy':['Comedy'],
    'crime': ['Crime'],
    'fantasy': ['Dark Fantasy','Dungeons and Dragons','Dungeons and Dragons (Game)'  , 'Epic','Esoterica', 'Fairies', 
                'Fairy Tales', 'Fantasy', 'Fantasy Of Manners','Folk Tales','Folklore', 'Heroic Fantasy' , 'Magical Realism', 
                    'Sci Fi Fantasy', 'Science Fiction Fantasy', 'Shapeshifters', 'Superheroes', 'Witchcraft', 
                    'World Of Warcraft', ],
    'fiction': ['Adult Fiction','Adventure','American fiction','Biblical Fiction','Brothers and sisters','Christian Fiction',
                 'Drama', 'Fan Fiction', 'Fantasy fiction', 'Fiction', 'Good and evil' , 'Historical Fiction', 
                   'Italian drama', 'Juvenile Fiction', 'Literary Fiction', 'Media Tie In', 'Military Fiction', 
                     'Plays','Pulp', 'Realistic Fiction', 'Retellings', 'Role Playing Games', 'School Stories', 
                       'Short Stories', 'Speculative Fiction', 'Tragedy', 'Weird Fiction', 'Westerns', 'Womens Fiction', 
                             ],
    'sexuality_and_gender': ['GLBT','Gay', 'Gender', 'Gender and Sexuality', 'LGBT', 'Lesbian', 'Polyamory', 'Queer', 
                             'Relationships', 'Sex', 'Sex Work', 'Sexuality', 'Womens', 'Womens Studies',  ],
    'literature': ['African Literature','American literature','Anthologies','Asian Literature','Criticism', 'European Literature',
                   'LITERARY CRITICISM','Literary Collections', 'Literary Criticism', 'Literature', 'Modern', 'Nobel Prize', 
                   'Novella', 'Novels',  ],
    'graphic_Novels': ['Graphic Novels', ],
    'historical': ['Historical', ],
    'history': ['1961-1975','American Civil War','American History','American Revolution','Antiques','Civil War',
                 'English History', 'European History', 'Frontier and pioneer life', 'Futurism','German History',  
                   'History', 'History and Politics', 'Medievalism', 'Military History', 'Roman', 'Russian History',  
                     'Spanish History', 'Stone age',  'Vietnam War', 'World War II',  ],
    'horror': ['Death', 'Horror', 'Horror tales', ],
    'magna': ['Comics Manga','Graphic Novels Manga', 'Manga', ],
    'memoir': ['Biography Memoir',],
    'music': ['Music', ],
    'mystery': ['Gothic', 'Mystery', ],
    'nonfiction': ['American','Anthropology','Archaeology','Astronomy','Academic', 'Artificial Intelligence', 'Aviation',
                    'Canon','Cartography', 'Chinese Americans','Cities', 'Collectibles', 'Conservation', 'Crafts', 
                    'Disability', 'Disciplines', 'Education', 'Emergency Services','Etiquette', 'European', 'Family', 
                    'Family Law', 'Games', 'Holiday', 'Home', 'Homeschool', 'House', 'Horticulture', 'Humanities', 
                    'Illness', 'Jewellery', 'Juvenile Nonfiction', 'Labor', 'Law', 'Linguistics', 'Mathematics', 
                      'Military', 'Mountaineering', 'Museology', 'Native Americans', 'Mythology', 'Neurodiversity',
                        'Nonfiction',  'Number', 'Outdoors', 'Parenting', 'Pets', 'Own','Politics', 'Pop Culture', 
                          'Presidents', 'Race', 'Railways', 'Reference', 'Recreation', 'Reportage', 'School', 'Social', 
                            'Social Issues', 'Social Justice', 'Social Movements', 'Sociology', 'Swedish', 'Teaching', 
                              'Television', 'Textbooks', 'True Story', 'Unfinished', 'Urban', 'War', 'Warriors', 'Writing', 
                                ],
    'paranormal': ['Paranormal', 'Occult', 'Surreal', ],
    'philosophy': ['Anarchism','Feminism', 'Philosophy',],
    'poetry': ['Childrens poetry', 'Poetry', 'Poetry Plays', ],
    'psychology': ['Mental Health', 'Mental health.', 'Mind', 'Minimal brain dysfunction in children', 'Psychology', 
                     ],
    'religion': ['Bibles','Biblical','Buddhism','Christian','Christianity', 'Church', 'Judaism', 'Lds', 'Prayer', 
                  'Religion', ],
    'romance':['Category Romance','Contemporary Romance', 'Erotica','Harlequin','Historical Romance', 'Love', 
               'Lovecraftian', 'Marriage', 'Menage', 'Romantic', 'Yaoi'  ],
    'science':['Animals','Bears','Biology', 'Computer Science', 'Computer software', 'Computers', 'Doctors', 
               'Earth', 'Engineering','Environment' , 'Gardening', 'Geography', 'Geology', 'Health', 'Internet',
                'Library Science', 'Medical', 'Medical Professions', 'Medicine', 'Nature', 'Neuroscience', 'Nursing', 
                  'Physics', 'Planets', 'Plants', 'Political Science', 'Pseudoscience', 'Science', 'Science Nature', 
                      'Social Science', 'Space', 'Study Aids', 'Technology', 'Wildlife',   ],
    'science_fiction': ['40k', 'Apocalyptic', 'Futuristic', 'Hugo Awards', 'Mmorpg', 'Science Fiction', 'Singularity', 
                        'Star Trek', ],
    'self_help': ['Activities', 'Body', 'Book Club', 'Fitness', 'Hobbies','How To', 'Inspirational', 'Leadership', 
                   'Personal', 'Productivity', 'Self Help',  ],
    'suspense': ['Dark', 'Suspense', ],
    'spirituality': ['Spirit', 'Spiritual life', 'Spirituality',  ],
    'sports': ['Football', 'Soccer', 'Sports','Sports and Games',  ],
    'thriller': ['Action', 'Combat', 'Spy Thriller', 'Thriller',  ],
    'travel': ['Africa','Athens (Greece)', 'Central Africa','Eastern Africa','India', 'New York', 'Northern Africa', 
               'Southern Africa', 'The Americas',  'The United States Of America', 'Transport', 'Transportation', 
                 'Travel', 'United States', 'Western Africa', ],
    'young_adult': ['Young Adult'],
}




#this dataframe is going to hold the combined genres columns
combined_df = pd.DataFrame()

#traverse the column mappings from new column name to old column name 
for new_column, old_columns in column_mapping.items():
    #sum up the values of the old column names and set those values equal to the values of the new column  (as stated above, 
    #this ensure that the column values will hold positive integers if they are a part of the corresponding genre)
    combined_df[new_column] = grdwg[old_columns].sum(axis=1)

#these are the columns that I wish to keep in the origional array 
    #this also serves the double purpose of removing columns that I do not want to keep in the dataset
    #I removed book_id, isbn13, language_code, and the origional genres column
      #book_id -> serves no purpose for my analysis
      #isbn13 -> i am keeping isbn if i need to use the isbn
      #language_code -> I do not need this in my analysis
      #genres -> this has been transformed into mulitple columns and the corresponding data preserved elsewhere... this columns 
      #data is now duplicated and no longer needed
columns_to_keep = ['author', 'average_rating',  'isbn', 'num_pages', 
                   'publication_date', 'publisher', 'ratings_count', 'text_reviews_count', 'title']


#drop the columns from the origional dataframe except for the columns that we want to keep in the dataframe
columns_to_drop = [col for col in grdwg.columns if col not in columns_to_keep]
grdwg.drop(columns=columns_to_drop, inplace=True)

#add the combined genres back to the origional dataframe
grdwg[combined_df.columns] = combined_df
#grdwg.head(10)

In [34]:
#NOW if a column for a certain genre contains a positive integer, it is a member of that genre. SO, to standardize everything
#if there is a value greater than 0 in the column, then cahnge it to 1. otherwise leave it as 0
#traverse the columns in the df
for col in grdwg.columns:
    #if the column is not in the previously defined columns to keep, then it is a genre column
    if col not in columns_to_keep:
        #if the value is greater than 0, change the value to 1 (essentially turning the column into a boolean )
        grdwg[col] =grdwg[col].apply(lambda x: 1 if x > 0 else 0)
#I also want the columns to be in the specified order. this basically creates a frame of the columns i want in the specified order
#and then tacks on the rest of the origional dataframe that did not contain those values 
new_column_order = ['title','author', 'num_pages','average_rating', 'ratings_count', 'text_reviews_count', 'publisher',
                    'publication_date', 'isbn'] + [col for col in grdwg.columns if col not in ['title', 'author', 'num_pages', 
                    'average_rating', 'ratings_count','text_reviews_count', 'publisher', 'publication_date', 'isbn']]
grdwg = grdwg[new_column_order]

#grdwg.head(10)

In [35]:
#grdwg.info()

In [36]:
#duplicate_rows = grdwg.duplicated()
#print("number of duplicate rows:" , duplicate_rows.sum())

In [37]:
#grdwg.nunique()

In [38]:
#this turns the dataframe into a new csv file and places it in the current folder
grdwg.to_csv('data.csv', index=False)